## 필요사항 설치

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 4.9 MB/s 
     |████████████████████████████████| 7.6 MB 82.5 MB/s 
     |████████████████████████████████| 163 kB 86.2 MB/s 


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 4.7 MB/s 
     |████████████████████████████████| 115 kB 89.1 MB/s 
     |████████████████████████████████| 212 kB 89.5 MB/s 
     |████████████████████████████████| 127 kB 92.3 MB/s 
     |████████████████████████████████| 115 kB 91.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
!pip install wandb -qU

     |████████████████████████████████| 1.9 MB 4.7 MB/s 
     |████████████████████████████████| 182 kB 72.9 MB/s 
     |████████████████████████████████| 166 kB 89.7 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 166 kB 88.4 MB/s 
     |████████████████████████████████| 162 kB 83.0 MB/s 
     |████████████████████████████████| 162 kB 95.4 MB/s 
     |████████████████████████████████| 158 kB 91.8 MB/s 
     |████████████████████████████████| 157 kB 99.0 MB/s 
     |████████████████████████████████| 157 kB 92.5 MB/s 
     |████████████████████████████████| 157 kB 93.0 MB/s 
     |████████████████████████████████| 157 kB 96.6 MB/s 
     |████████████████████████████████| 157 kB 89.4 MB/s 
     |████████████████████████████████| 157 kB 90.8 MB/s 
     |████████████████████████████████| 157 kB 93.1 MB/s 
     |████████████████████████████████| 156 kB 90.9 MB/s 


In [4]:
!kaggle datasets download -d minjaechoi99/aihub-groom

 97% 37.0M/38.1M [00:00<00:00, 210MB/s] 
100% 38.1M/38.1M [00:00<00:00, 187MB/s]


In [5]:
!unzip /content/aihub-groom.zip

Archive:  /content/aihub-groom.zip
  inflating: data/Training/talksets-train-1.json  
  inflating: data/Training/talksets-train-2.json  
  inflating: data/Training/talksets-train-3.json  
  inflating: data/Training/talksets-train-4.json  
  inflating: data/Training/talksets-train-5.json  
  inflating: data/Validation/talksets-train-6.json  


## 데이터 전처리

In [6]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
import torch
import torch.optim as optim

import numpy as np
import tqdm as tqdm

from transformers import(
    AutoModel,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    default_data_collator,
    EarlyStoppingCallback,
    get_cosine_schedule_with_warmup,
    get_linear_schedule_with_warmup,
    get_constant_schedule,
    DefaultDataCollator,
    AdamW
)

## readjson

In [8]:
from pathlib import Path
import json, re
def read_aihub(path):
    path = Path(path)
    with open(path, 'rb') as f:
        print(f)
        squad_dict = json.load(f)
    texts = []
    intensities = []
    for group in squad_dict:
        for passage in group['sentences']:
            text = passage['origin_text']
            text = re.sub('#.*?#', '[UNK]', text)
            intensity = passage['intensity']
            if intensity == 1:
                continue
            texts.append(text)
            if intensity < 1:
                intensity = 0
            else:
                intensity = 1
            intensities.append(intensity)

    return texts ,intensities

In [9]:
import pandas as pd

file_path = '/content/data/Validation/talksets-train-6.json'

texts , intensities = read_aihub(file_path)
test_df = pd.DataFrame(list(zip(texts , intensities)),
               columns =['texts','intensities'])

<_io.BufferedReader name='/content/data/Validation/talksets-train-6.json'>


In [10]:
from datasets import Dataset

test_data = Dataset.from_pandas(test_df)

In [11]:
test_data

Dataset({
    features: ['texts', 'intensities'],
    num_rows: 37319
})

## tokenize

In [12]:
model_name = 'beomi/KcELECTRA-base-v2022'
max_length = 128

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/504 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [14]:
def tokenizeWithoutLabel(data):
    tokenized_datas = tokenizer(
        data['texts'],
        max_length=max_length,
        padding="max_length",
        truncation="only_second"
    )
    return tokenized_datas

In [15]:
test_tokenized_datasets = test_data.map(tokenizeWithoutLabel, batched=True, remove_columns=test_data.column_names)

  0%|          | 0/38 [00:00<?, ?ba/s]

In [16]:
test_tokenized_datasets

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 37319
})

## test

In [33]:
import wandb
run = wandb.init()
artifact1 = run.use_artifact('groom2team/aihub_pj3/pytorch_finetuned:v0', type='model')
artifact_dir1 = artifact1.download()

wandb: Downloading large artifact pytorch_finetuned:v0, 489.19MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:5.5


In [34]:
torch.cuda.empty_cache()

In [35]:
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.load_state_dict(torch.load(artifact_dir1+'/best_model_at_end/pytorch_model.bin'))

Some weights of the model checkpoint at beomi/KcELECTRA-base-v2022 were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.out_proj.bias',

<All keys matched successfully>

In [36]:
data_collator = DefaultDataCollator(return_tensors="pt")
test_loader = torch.utils.data.DataLoader(test_tokenized_datasets, batch_size=64,
                                        shuffle=False, collate_fn=data_collator,
                                        num_workers=0)

In [37]:
for data in test_loader:
    print(data)
    break

{'input_ids': tensor([[    2, 14680,  8178,  ...,     0,     0,     0],
        [    2,  8846, 15901,  ...,     0,     0,     0],
        [    2, 12063,  2607,  ...,     0,     0,     0],
        ...,
        [    2, 10288, 14812,  ...,     0,     0,     0],
        [    2,  2576,  4810,  ...,     0,     0,     0],
        [    2, 17174,  1836,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [38]:
device = torch.device('cuda')

In [39]:
from tqdm.auto import tqdm
def get_prob(model, test_loader):
    model = model.to(device)
    pred = []
    error = 0
    special_tokens = set(tokenizer.all_special_ids)
    softmax = torch.nn.Softmax()
    for i, id_inputs in enumerate(tqdm(test_loader)):
        torch.cuda.empty_cache()
        inputs = {}
        inputs['input_ids'] = id_inputs['input_ids'].to(device)
        inputs['token_type_ids'] = id_inputs['token_type_ids'].to(device)
        inputs['attention_mask'] = id_inputs['attention_mask'].to(device)
        outputs = model(**inputs)
        pred += [0 if logit[0] > logit[1] else 1 for logit in outputs.logits]
    return pred

In [40]:
pred = get_prob(model, test_loader)

  0%|          | 0/584 [00:00<?, ?it/s]

In [41]:
len(pred)

37319

In [42]:
predictions = test_df.drop(['texts'], axis = 1)
predictions['pred'] = pred
predictions.rename(columns = {'intensities':'answer'},inplace=True)

In [43]:
predictions

,answer,pred
0,0,0
1,1,1
2,0,0
3,0,0
4,1,0
...,...,...
37314,0,0
37315,1,1
37316,0,0
37317,1,1


In [46]:
tp, tn, fp, fn = (0, 0, 0, 0)

In [47]:
for index, row in predictions.iterrows():
    # 0이 negative 1이 positive
    ans, pred = (row["answer"], row["pred"])
    if ans == 0:
        if pred == 0:
            tn += 1
        else:
            fp += 1
    else:
        if pred == 0:
            fn += 1
        else:
            tp += 1

In [48]:
(tp+tn)/(tp+tn+fp+fn)

0.8793108068276213

In [31]:
predictions.to_csv('predictions.csv', index = False)